# **Proyecto final - Seguridad**

## **Universidad Icesi**

### **Santiago Rodas Rodriguez**

El presente proyecto consiste en realizar un simulador de una maquina enigma, implementando un login cifrado donde se administrara una serie de usuarios. Esos usuarios allí agregados son los que tendrán la capacidad de utilizar la maquina enigma con todas sus funcionalidades.

Tomando en cuenta lo anterior, el proyecto consta de 3 partes fundamentales:

1. Login cifrado usando paquetes

2. Implementación de la maquina enigma

3. Flujo y administración de usuarios

## **Parte 1: Login cifrado usando paquetes**

In [127]:
# Instalamos los paquetes necesarios para el login y comprobamos su uso

!pip install werkzeug
!pip install passlib
!pip install cryptography

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [128]:
#Importamos todas las librerias necesarias para nuestro login

from werkzeug.security import generate_password_hash, check_password_hash #Generar encriptacion
from werkzeug.security import check_password_hash #Validar los valores cifrados
from passlib.context import CryptContext #Texto para hacer encriptacion
from cryptography.fernet import Fernet

import pandas as pd
import sys

### Capa 1

In [129]:
# Esta funcion nos permite cifrar un texto por medio del paquete werkzeug

def cifrarTexto1(password):
    return generate_password_hash(password, 'pbkdf2:sha256', 1000)

In [130]:
# Esta funcion nos permite comprobar la informacion del usuario

def descifrarTexto1(cifrado, password):
    return check_password_hash(cifrado, password)

### Capa 2

In [131]:
# Esta funcion nos permite cifrar un texto por medio del paquete passlib

def cifrarTexto2(password):
    contexto = CryptContext(schemes=["des_crypt"])
    return contexto.hash(password)

In [132]:
# Esta funcion nos permite comprobar la informacion del usuario

def descifrarTexto2(password, cifrado):
    return contexto.verify(password, cifrado)

### Capa 3

In [133]:
key = Fernet.generate_key()
llave_cifrada = Fernet(key)

In [134]:
# Esta funcion nos permite cifrar un texto por medio del paquete Fernet

def cifrarTexto3(password):
    return llave_cifrada.encrypt(str.encode(password))

In [135]:
# Esta funcion nos permite comprobar la informacion del usuario

def descifrarTexto3(cifrado):
    bytes = llave_cifrada.decrypt(cifrado)
    return bytes.decode()

## **Parte 2: Implementación de la maquina enigma**

Vamos a simular la máquina Enigma usando Python. Para empezar, vamos a simplificar la máquina como una serie de permutaciones que nos envían letras del alfabeto a otras letras.

Esto lo usaremos para pasar letras a números, y así poder buscarlas en nuestras cadenas que usaremos para representar las permutaciones.

In [136]:
def letra_a_numero(a):
  return ord(a.upper()) - ord("A")

In [137]:
def numero_a_letra(a):
  return "ABCDEFGHIJKLMNOPQRSTUVWXYZ"[a]

### Configuramos los rotores

La máquina que estamos emulando es el modelo E-Eins, o sea, la primera de uso militar. Esta máquina tiene tres rotores, a elegir entre cinco, situándose entre los contactos de entrada/salida y el llamado reflector.

Cuando pulsamos una tecla del teclado de la máquina, el rotor de la derecha gira una posición, y después se cierra un circuito que pasa por cada disco, luego vuelve por el reflector para luego volver por los rotores y llegar a una bombilla asignada a una letra, que será la letra cifrada.

In [138]:
def ciclo_de_permutacion(permuta, n = 1):
  for _ in range(0, n):
    permuta = permuta[1:] + str(permuta[0])
    permuta = "".join([numero_a_letra((letra_a_numero(i)-1) % 26) for i in permuta])
  return permuta

In [139]:
def realiza_una_permutacion(permuta, direccion, a):
    if direccion == 1:
        return permuta[letra_a_numero(a)]
    else:
        return numero_a_letra(permuta.index(a))

Cada rotor tiene dos configuraciones: la configuración interna, que es un desfase del disco indicador del rotor con respecto al cableado interno y la configuración externa, que es el desfase del rotor una vez introducido.

In [140]:
class Rotor:

    # Inicializa cada rotor

    def __init__(self, to, perm):
        self.configuracion_intera = 0
        self.configuracion_externa = 0
        self.permutacion_inicial = perm
        self.to = to # Donde se hace la rotacion
        self.perm = perm # Permutación del anillo
        self.esRotacion = False # Si se tiene o no que hacer la rotacion

    # Cambia la configuracion interna

    def set_configuracion_intera(self, configuracion_intera):
        self.configuracion_intera = configuracion_intera
        self.perm = ciclo_de_permutacion(self.perm, - configuracion_intera % 26)
    
    # Cambia la configuracion externa

    def set_configuracion_externa(self, configuracion_externa):
        self.configuracion_externa = configuracion_externa
        self.perm = ciclo_de_permutacion(self.permutacion_inicial, configuracion_externa)
    
    # Realiza una permutacion de ida

    def permutacion_ida(self, a):
        return realiza_una_permutacion(self.perm, 1, a)
    
    # Realiza una permutacion de vuelta

    def permutacion_vuelta(self, a):
        return realiza_una_permutacion(self.perm, 0, a)
    
    # Realiza un giro

    def giro(self):
        self.configuracion_externa = (self.configuracion_externa + 1) % 26
        self.perm = ciclo_de_permutacion(self.perm)
        if (self.configuracion_externa) % 26 == self.to:
            self.esRotacion = True
    
    # Realiza una permutacion

    def rotacion(self):
        if self.esRotacion:
            self.esRotacion = False
            return True
        else:
            return False

### Configuramos el clavijero

El clavijero era una fase extra de seguridad que permutaba una letra por otra a la entrada y salida.

In [141]:
def define_un_clavijero(pares, a):
    if pares == None:
        return a
    elif not (a in pares.keys() or a in pares.values()):
        return a
    else:
        for k, v in pares.items():
            if k == a:
                return v
            elif v == a:
                return k

### Configuramos la maquina

In [142]:
class Maquina:

    # Inicializa la maquina

    def __init__(self, r1, r2, r3, ukw, kl):
        self.r1, self.r2, self.r3 = r1, r2, r3
        self.ukw = ukw
        self.kl = kl

    # Configuraciones
        
    def configuraciones(self, gss, rss):
        self.r1.set_configuracion_externa(gss[2])
        self.r2.set_configuracion_externa(gss[1])
        self.r3.set_configuracion_externa(gss[0])
        
        self.r1.set_configuracion_intera(rss[2])
        self.r2.set_configuracion_intera(rss[1])
        self.r3.set_configuracion_intera(rss[0])

    # Cifra una letra especifica

    def cifrar_una_letra(self, a):
        self.r1.giro()
        if self.r1.rotacion():
            self.r2.giro()
            if self.r2.rotacion():
                self.r3.giro()

        ret = define_un_clavijero(self.kl, a)
        ret = self.r1.permutacion_ida(ret)
        ret = self.r2.permutacion_ida(ret)
        ret = self.r3.permutacion_ida(ret)
        ret = realiza_una_permutacion(self.ukw,1,ret)

        ret = self.r3.permutacion_vuelta(ret)
        ret = self.r2.permutacion_vuelta(ret)
        ret = self.r1.permutacion_vuelta(ret)
        ret = define_un_clavijero(self.kl, ret)
        return ret
    
    # Cifra el mensaje completo

    def cifrar_mensaje_completo(self,X):
        return "".join([self.cifrar_una_letra(i) for i in X])
    
    # Vemos la configuracion externa de los rotores

    def __repr__(self):
        return "(" + numero_a_letra(M.r3.configuracion_externa) + ", " + numero_a_letra(M.r2.configuracion_externa) + ", " + numero_a_letra(M.r1.configuracion_externa) + ")"

### Cableados

Generamos los cableados que se usaban en la guerra.

In [143]:
permR1 = "EKMFLGDQVZNTOWYHXUSPAIBRCJ"
permR2 = "AJDKSIRUXBLHWTMCQGZNPYFVOE"
permR3 = "BDFHJLCPRTXVZNYEIWGAKMUSQO"
permR4 = "ESOVPZJAYQUIRHXLNFTGKDCMWB"
permR5 = "VZBRGITYUPSDNHLXAWMJQOFECK"

UKW_B  = "YRUHQSLDPXNGOKMIEBFZCWVJAT"
UKW_C  = "FVPJIAOYEDRZXWGCTKUQSBNMHL"

Generamos los rotores.

In [144]:
R1 = Rotor(letra_a_numero("Q")+1, permR1)
R2 = Rotor(letra_a_numero("E")+1, permR2)
R3 = Rotor(letra_a_numero("V")+1, permR3)
R4 = Rotor(letra_a_numero("J")+1, permR3)
R5 = Rotor(letra_a_numero("Z")+1, permR3)

In [145]:
KL = {"A":"K","H":"L"}

M = Maquina(R3,R2,R1,UKW_B,KL)

GS = ["D","I","E"]

RS = ["G","O","V"]

M.configuraciones(list(map(letra_a_numero, GS)), list(map(letra_a_numero, RS)))

### Funcionalidades finales

In [146]:
def cifrar_en_maquina_enigma(texto):
    return M.cifrar_mensaje_completo(texto)

In [147]:
def descifrar_en_maquina_enigma(texto):
    M.configuraciones(list(map(letra_a_numero, GS)), list(map(letra_a_numero, RS)))
    return M.cifrar_mensaje_completo(texto)

## **Parte 3: Flujo y administración de usuarios**

In [148]:
# Inicializamos la informacion de los usuarios

df = pd.DataFrame()
df['username'] = None
df['capa1'] = None
df['capa2'] = None
df['capa3'] = None
df['original'] = None
nueva_fila = {'username' : "Santiago", 'capa1': cifrarTexto1("Rodas"), 'capa2' : cifrarTexto2("Rodas"), 'capa3' : cifrarTexto3("Rodas"), 'original' : "Rodas"}
df = df.append(nueva_fila, ignore_index = True)

In [149]:
# Metodo para verificar la informacion

def verificar_usuario(usuario, password):
    v1 = usuario in df.username.values
    v2 = cifrarTexto1(password) in df.capa1.values
    v3 = cifrarTexto2(password) in df.capa2.values
    v4 = cifrarTexto3(password) in df.capa3.values
    v5 = password in df.original.values

    if(v5 == True):
        return True
    else:
        return False


# --------------------------------------------------------------------------------

# Menu principal del programa

def menu_principal(usuario):

    print()
    print("Hola, ", usuario)
    print()

    validacion_menu = True

    while validacion_menu:
        print()
        print("Seleccione una opcion")
        print("1. Agregar usuario")
        print("2. Utilizar la maquina enigma")
        print("3. Salir del sistema")
        print()
        numero = int(input())

        if numero == 1:

            agregar_usuario()
        
        if numero == 2:

            utilizar_maquina()
        
        if numero == 3:

            validacion_menu = False
            print()
            print("Sistema desactivado")
            print()
            sys.exit()

# --------------------------------------------------------------------------------

# Utilizar la maquina enigma

def utilizar_maquina():

    enigma_validacion = True

    while enigma_validacion:

        print()
        print("Opcion 1 - Cifrar informacion")
        print("Opcion 2 - Descifrar informacion")
        print("Opcion 3 - Volver al menu principal")
        print()
        opcion_usuario = int(input())

        if opcion_usuario == 1:
            
            print("Ingrese la cadena de texto que desea validar en la maquina Enigma")
            texto_maquina = input()
            texto_maquina_utilizar = texto_maquina.upper()
            print()

            nuevo_texto_maquina = texto_maquina_utilizar.replace(" ", "X")
            print("El cifrado es:")
            print(cifrar_en_maquina_enigma(nuevo_texto_maquina))
            print()

        if opcion_usuario == 2:

            print()
            print("Ingrese el cifrado que desea obtener la informacion original")
            texto_maquina2 = input()
            print()

            print()
            print("La informacion original es:")
            nuevo_texto_maquina2 = descifrar_en_maquina_enigma(texto_maquina2)
            print(nuevo_texto_maquina2.replace("X", " "))
            print()
        
        if opcion_usuario == 3:

            enigma_validacion = False

# --------------------------------------------------------------------------------

# Agregar usuario al sistema

def agregar_usuario():
    
    print()
    print("Ingrese el nuevo nombre de usuario:")
    nuevo_usuario = input()
    print()

    print("Ingrese la nueva contraseña de usuario:")
    nueva_password = input()
    print()

    df_nuevo = pd.DataFrame()
    df_nuevo['username'] = None
    df_nuevo['capa1'] = None
    df_nuevo['capa2'] = None
    df_nuevo['capa3'] = None
    df_nuevo['original'] = None

    nueva_fila = {'username' : nuevo_usuario, 'capa1': cifrarTexto1(nueva_password), 'capa2' : cifrarTexto2(nueva_password), 'capa3' : cifrarTexto3(nueva_password), 'original' : nueva_password}
    df_nuevo = df_nuevo.append(nueva_fila, ignore_index = True)

    print("Usuario agregado correctamente")
    print()

    print(df_nuevo.to_string())
    print()

In [150]:
# Flujo de usuarios y administracion de estos mismos

print("Bienvenido a la maquina enigma")
print("Por favor inicie sesion dentro del sistema, tiene 3 intentos")

contador_iniciar_sesion = 0

while contador_iniciar_sesion <= 2:

    print()
    print("Por favor escriba el usuario:")
    usuario_legal = input()

    print("Por favor escriba la contraseña:")
    password_legal = input()

    validacion_final = verificar_usuario(usuario_legal, password_legal)

    if validacion_final == True:

        menu_principal(usuario_legal)

    else:
    
        contador_iniciar_sesion += 1
        print()
        print("Informacion erronea")

Bienvenido a la maquina enigma
Por favor inicie sesion dentro del sistema, tiene 3 intentos

Por favor escriba el usuario:
Santiago
Por favor escriba la contraseña:
Rodas

Hola,  Santiago


Seleccione una opcion
1. Agregar usuario
2. Utilizar la maquina enigma
3. Salir del sistema

2

Opcion 1 - Cifrar informacion
Opcion 2 - Descifrar informacion
Opcion 3 - Volver al menu principal

1
Ingrese la cadena de texto que desea validar en la maquina Enigma
HOLA COMO ESTAS

El cifrado es:
USCEBNNXBMQZWYU


Opcion 1 - Cifrar informacion
Opcion 2 - Descifrar informacion
Opcion 3 - Volver al menu principal

2

Ingrese el cifrado que desea obtener la informacion original
USCEBNNXBMQZWYU


La informacion original es:
HOLA COMO ESTAS


Opcion 1 - Cifrar informacion
Opcion 2 - Descifrar informacion
Opcion 3 - Volver al menu principal



KeyboardInterrupt: ignored

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4aaa3fa0-ff42-4b17-a433-b4058bf91fd5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>